In [1]:
from qiskit.tools.jupyter import *
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')

In [2]:
backend_config = backend.configuration()
assert backend_config.open_pulse, "Backend doesn't support Pulse"

In [3]:
dt = backend_config.dt
print(f"Sampling time: {dt*1e9} ns")    # The configuration returns dt in seconds, so multiply by
                                        # 1e9 to get nanoseconds

Sampling time: 0.2222222222222222 ns


In [4]:
backend_defaults = backend.defaults()

In [5]:
import numpy as np

# unit conversion factors -> all backend properties returned in SI (Hz, sec, etc)
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds

# We will find the qubit frequency for the following qubit.
qubit = 0

center_frequency_Hz = backend_defaults.qubit_freq_est[qubit]  

# scale factor to remove factors of 10 from the data
scale_factor = 1e-14

# We will sweep 40 MHz around the estimated frequency
frequency_span_Hz = 40 * MHz
# in steps of 1 MHz.
frequency_step_Hz = 0.8 * MHz

# We will sweep 20 MHz above and 20 MHz below the estimated frequency
frequency_min = center_frequency_Hz - frequency_span_Hz / 2
frequency_max = center_frequency_Hz + frequency_span_Hz / 2
# Construct an np array of the frequencies for our experiment
frequencies_GHz = np.arange(frequency_min / GHz, 
                            frequency_max / GHz, 
                            frequency_step_Hz / GHz)

In [6]:
# samples need to be multiples of 16
def get_closest_multiple_of_16(num):
    return int(num + 8 ) - (int(num + 8 ) % 16)

In [7]:
from qiskit import pulse            # This is where we access all of our Pulse features!
from qiskit.pulse import Play
from qiskit.pulse import pulse_lib  # This Pulse module helps us build sampled pulses for common pulse shapes


# Drive pulse parameters (us = microseconds)
drive_sigma_us = 0.075                     # This determines the actual width of the gaussian
drive_samples_us = drive_sigma_us*8        # This is a truncating parameter, because gaussians don't have 
                                           # a natural finite length

drive_sigma = get_closest_multiple_of_16(drive_sigma_us * us /dt)       # The width of the gaussian in units of dt
drive_samples = get_closest_multiple_of_16(drive_samples_us * us /dt)   # The truncating parameter in units of dt
drive_amp = 0.3
# Drive pulse samples
drive_pulse = pulse_lib.gaussian(duration=drive_samples,
                                 sigma=drive_sigma,
                                 amp=drive_amp,
                                 name='freq_sweep_excitation_pulse')

In [8]:
# Find out which group of qubits need to be acquired with this qubit
meas_map_idx = None
for i, measure_group in enumerate(backend_config.meas_map):
    if qubit in measure_group:
        meas_map_idx = i
        break
assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"

In [9]:
inst_sched_map = backend_defaults.instruction_schedule_map
measure = inst_sched_map.get('measure', qubits=backend_config.meas_map[meas_map_idx])

In [10]:
### Collect the necessary channels
drive_chan = pulse.DriveChannel(qubit)
meas_chan = pulse.MeasureChannel(qubit)
acq_chan = pulse.AcquireChannel(qubit)

In [11]:
# Create the base schedule
# Start with drive pulse acting on the drive channel
schedule = pulse.Schedule(name='Frequency sweep')
schedule += Play(drive_pulse, drive_chan)
# The left shift `<<` is special syntax meaning to shift the start time of the schedule by some duration
schedule += measure << schedule.duration

# Create the frequency settings for the sweep (MUST BE IN HZ)
frequencies_Hz = frequencies_GHz*GHz
schedule_frequencies = [{drive_chan: freq} for freq in frequencies_Hz]

In [12]:
from qiskit import assemble

num_shots_per_frequency = 1024
frequency_sweep_program = assemble(schedule,
                                   backend=backend, 
                                   meas_level=1,
                                   meas_return='avg',
                                   shots=num_shots_per_frequency,
                                   schedule_los=schedule_frequencies)

In [13]:
job = backend.run(frequency_sweep_program)

In [14]:
# print(job.job_id())
from qiskit.tools.monitor import job_monitor
job_monitor(job)

Job Status: job has successfully run


In [15]:
frequency_sweep_results = job.result(timeout=120) # timeout parameter set to 120 seconds

In [16]:
import matplotlib.pyplot as plt

sweep_values = []
for i in range(len(frequency_sweep_results.results)):
    # Get the results from the ith experiment
    res = frequency_sweep_results.get_memory(i)*scale_factor
    # Get the results for `qubit` from this experiment
    sweep_values.append(res[qubit])

#plt.scatter(frequencies_GHz, np.real(sweep_values), color='black') # plot real part of sweep values
#plt.xlim([min(frequencies_GHz), max(frequencies_GHz)])
#plt.xlabel("Frequency [GHz]")
#plt.ylabel("Measured signal [a.u.]")
#plt.show()

In [17]:
from scipy.optimize import curve_fit

def fit_function(x_values, y_values, function, init_params):
    fitparams, conv = curve_fit(function, x_values, y_values, init_params)
    y_fit = function(x_values, *fitparams)
    
    return fitparams, y_fit

In [18]:
fit_params, y_fit = fit_function(frequencies_GHz,
                                 np.real(sweep_values), 
                                 lambda x, A, q_freq, B, C: (A / np.pi) * (B / ((x - q_freq)**2 + B**2)) + C,
                                 [5, 4.975, 1, 3] # initial parameters for curve_fit
                                )

In [19]:
#plt.scatter(frequencies_GHz, np.real(sweep_values), color='black')
#plt.plot(frequencies_GHz, y_fit, color='red')
#plt.xlim([min(frequencies_GHz), max(frequencies_GHz)])

#plt.xlabel("Frequency [GHz]")
#plt.ylabel("Measured Signal [a.u.]")
#plt.show()

In [20]:
#print(frequencies_GHz)
#print(np.real(sweep_values))

In [21]:
guesses = np.array([])
guess_results = np.array([])

In [22]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [23]:
all_freq = []
all_freq_options = []
for i in range(len(frequencies_GHz)):
    all_freq.append([round(frequencies_GHz[i],3),np.real(sweep_values)[i]])
    all_freq_options.append(str(round(frequencies_GHz[i],3)))

In [24]:
#print(all_freq_options)
#print(all_freq)

In [25]:
def guess_signal(Guess):
    global guesses
    global guess_results
    for i in range(len(all_freq_options)):
        if all_freq_options[i] == Guess:
            guesses = np.append(guesses, [all_freq[i][0]])
            print(all_freq[i][0])
            print(guesses)
            guess_results = np.append(guess_results, [all_freq[i][1]])
            print(guess_results)

            plt.scatter(guesses, guess_results, color='black')
            plt.xlim([min(guesses), max(guesses)])

            plt.xlabel("Guessed Frequency [GHz]")
            plt.ylabel("Measured Signal [a.u.]")
            plt.show()

In [26]:
interact_manual(guess_signal, Guess = all_freq_options)

interactive(children=(Dropdown(description='Guess', options=('4.954', '4.955', '4.956', '4.957', '4.958', '4.9…

<function __main__.guess_signal(Guess)>

In [27]:
print(guesses)
print(guess_results)

[]
[]
